添加Coupon_all_consume特征，标记那些领取优惠劵数目等于使用优惠劵数目为1，否则为0

In [ ]:
user_coupon = off_train[['User_id', 'Coupon_id']]
# 利用User_id和Coupon_id两列对用户进性分组，并且计算每组的数目，并进行序号重置
user_coupon_count = user_coupon.groupby(by=['User_id']).size().reset_index()
# 重命名
user_coupon_count.rename(columns={0:'Coupon_size'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge Coupon_size代表用户领取的优惠劵数目
off_train = off_train.merge(user_coupon_count, on='User_id', how='inner')

user_coupon = off_train[['User_id', 'result']]
# 利用User_id和result两列对用户进性分组，并且计算每组的数目（这里相当于求和），并进行序号重置
user_coupon_used_count = user_coupon.groupby(by=['User_id']).sum().reset_index()
# 重命名 Consume_count用户使用了的优惠劵数目
user_coupon_used_count.rename(columns={'result':'Conupon_have_used_count'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge
off_train = off_train.merge(user_coupon_used_count, on='User_id', how='inner')

# 为训练集添加Coupon_all_consume特征，标记那些领取优惠劵数目等于使用优惠劵数目
off_train['Coupon_all_consume'] = off_train[['Conupon_have_used_count','Coupon_size']].apply(lambda x : 1 if x[0] == x[1] else 0, axis = 1)
# 去掉产生的中间列
off_train = off_train.drop(columns=['Coupon_size', 'Conupon_have_used_count'])
# 提取需要的列好merge到测试集中
off_train_coupon_all_consume = off_train[['User_id', 'Coupon_all_consume']]
off_train_coupon_all_consume = off_train_coupon_all_consume.drop_duplicates(['User_id', 'Coupon_all_consume'])
#为测试集添加Coupon_all_consume特征
off_test = off_test.merge(off_train_coupon_all_consume, on='User_id', how='left')
# 填充merge以后的空值为0
off_test = off_test.fillna({'Coupon_all_consume':0})